<h1 align="center"> Project Setup </h1>

#### Install all requirements for the project

In [1]:
print('--- Upgrading pip ---')
!pip install --upgrade pip
print('--- Cheking pip version ---')
!pip --version

--- Upgrading pip ---
--- Cheking pip version ---
pip 21.2.4 from /Users/kesnel/opt/anaconda3/lib/python3.8/site-packages/pip (python 3.8)


#### Importing Dependencies

In [2]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans

#### Importing Datasets

In [3]:
# Display all columns in the DataFrame using pandas settings
pd.set_option('display.max_columns', None)

In [11]:
df_imports = pd.read_excel('data/WtoData_worldwide_import_from_2010_to_2022.xlsx')

<br><br>

<h1 align="center"> Summary</h1>

### Objective

This project focuses on how the country will be able to know the right market to export their products. This will allow the country to get a good guide on foreign trade, and know which countries have High demand in different sectors, especially agricultural products.

The specific objectives of this project are:
1. Analyzethedatasetthatwearegoingtocollecttobetterunderstandandgive
some insight on the international trade
2. ChoosethebestplacewiththeHighdemandforagriculturalproductssothat Haiti can promote and sell its products

<hr>

### Hypothesis: Research Question?

What is the question that you would like to answer in order to make a decision.

<hr>

### Data Source

For this project we will be using the World Trade Organization(WTO) data portal to have access to. The WTO Data portal contains statistical indicators, Available time series cover merchandise trade and trade in services statistics, market access indicators (bound, applied & preferential tariffs), non-tariff information as well as other indicators.

<br><br>

<h1 align="center"> Data Cleaning </h1>

#### Data Overview

In [13]:
# Displaying the dataframe
print('------ Imports Dataset ------')
display(df_imports.head(4))

------ Imports Dataset ------


,Indicator,Merchandise imports by product group – annual (Million US dollar),Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Reporting Economy,Product/Sector,Partner Economy,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
2,World,SI3_AGG - TO - Total merchandise,World,15438092.0,18438364.0,18657296.0,18966119.0,19060809.0,16733507.0,16211194.0,17985896.0,19836342.0,19284167.0,17812107.0
3,World,SI3_AGG - AG - Agricultural products,World,1391529.0,1701893.0,1681611.0,1756648.0,1798003.0,1594510.0,1599989.0,1759978.0,1851346.0,1823105.0,NaN


In [14]:
print('------------------------------ Dataset Shape ------------------------------')
print('The Imports dataset has',df_imports.shape[0], 'Rows and', df_imports.shape[1],'columns')

print('------------------------------ Imports dataset Columns ------------------------------')
display(df_imports.columns)

print('------------------------------ Imports Data types ------------------------------')
display(df_imports.dtypes)

print('------------------------------ Imports dataframe info ------------------------------')
display(df_imports.info())

print('------------------------------ Imports dataframe info ------------------------------')
display(df_imports.describe())

------------------------------ Dataset Shape ------------------------------
The Imports dataset has 3461 Rows and 14 columns
------------------------------ Imports dataset Columns ------------------------------


Index(['Indicator',
       '  Merchandise imports by product group – annual (Million US dollar)',
       'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13'],
      dtype='object')

------------------------------ Imports Data types ------------------------------


Indicator                                                               object
  Merchandise imports by product group – annual (Million US dollar)     object
Unnamed: 2                                                              object
Unnamed: 3                                                             float64
Unnamed: 4                                                             float64
Unnamed: 5                                                             float64
Unnamed: 6                                                             float64
Unnamed: 7                                                             float64
Unnamed: 8                                                             float64
Unnamed: 9                                                             float64
Unnamed: 10                                                            float64
Unnamed: 11                                                            float64
Unnamed: 12                                         

------------------------------ Imports dataframe info ------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3461 entries, 0 to 3460
Data columns (total 14 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   Indicator                                                            3460 non-null   object 
 1     Merchandise imports by product group – annual (Million US dollar)  3460 non-null   object 
 2   Unnamed: 2                                                           3460 non-null   object 
 3   Unnamed: 3                                                           3313 non-null   float64
 4   Unnamed: 4                                                           3349 non-null   float64
 5   Unnamed: 5                                                           3303 non-null   float64
 6   Unnamed: 6         

None

------------------------------ Imports dataframe info ------------------------------


,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
count,3.313000e+03,3.349000e+03,3.303000e+03,3.283000e+03,3.272000e+03,3.288000e+03,3.274000e+03,3.307000e+03,3.308000e+03,3.308000e+03,2.610000e+02
mean,6.058404e+04,7.107347e+04,7.200046e+04,7.383176e+04,7.479535e+04,6.536848e+04,6.404206e+04,7.071302e+04,7.807287e+04,7.583848e+04,5.011247e+05
std,5.523083e+05,6.535578e+05,6.604516e+05,6.741555e+05,6.811720e+05,6.004346e+05,5.867944e+05,6.484204e+05,7.143228e+05,6.946708e+05,1.994239e+06
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.190000e+02,1.310000e+02,1.420000e+02,1.540000e+02,1.677500e+02,1.527500e+02,1.630000e+02,1.785000e+02,1.907500e+02,1.917500e+02,2.458000e+03
50%,8.840000e+02,1.022000e+03,1.092000e+03,1.158000e+03,1.241500e+03,1.119000e+03,1.081000e+03,1.151000e+03,1.308000e+03,1.290500e+03,1.479600e+04
75%,6.461000e+03,7.529000e+03,8.006500e+03,8.368500e+03,8.515500e+03,7.243250e+03,6.934250e+03,7.720500e+03,8.520750e+03,8.158000e+03,1.529940e+05
max,1.543809e+07,1.843836e+07,1.865730e+07,1.896612e+07,1.906081e+07,1.673351e+07,1.621119e+07,1.798590e+07,1.983634e+07,1.928417e+07,1.781211e+07


In [15]:
# Display all columns
print('-------- Default column names --------')
display(df_imports.columns)

# Change columns name
map_cols_name = {
    'Indicator':'Reporting Economy',
    '  Merchandise imports by product group – annual (Million US dollar)': 'Product/Sector',
    'Unnamed: 2': 'Partner Economy',
    'Unnamed: 3': '2010',
    'Unnamed: 4': '2011',
    'Unnamed: 5': '2012',
    'Unnamed: 6': '2013',
    'Unnamed: 7': '2014',
    'Unnamed: 8': '2015',
    'Unnamed: 9': '2016',
    'Unnamed: 10': '2017',
    'Unnamed: 11': '2018',
    'Unnamed: 12': '2019',
    'Unnamed: 13': '2020',
}

# Change all default column names
print('-------- With new column names --------')
renamed_cols_df = df_imports.rename(columns=map_cols_name)
display(renamed_cols_df.head(3))

-------- Default column names --------


Index(['Indicator',
       '  Merchandise imports by product group – annual (Million US dollar)',
       'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6',
       'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13'],
      dtype='object')

-------- With new column names --------


,Reporting Economy,Product/Sector,Partner Economy,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Reporting Economy,Product/Sector,Partner Economy,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0
2,World,SI3_AGG - TO - Total merchandise,World,15438092.0,18438364.0,18657296.0,18966119.0,19060809.0,16733507.0,16211194.0,17985896.0,19836342.0,19284167.0,17812107.0


In [40]:
# Remove the first 2 rows
to_drop_rows = renamed_cols_df.index[:2]
display(to_drop_rows)

dropped_rows_df = renamed_cols_df.drop(to_drop_rows).reset_index(drop=True)
display(dropped_rows_df.head())

RangeIndex(start=0, stop=2, step=1)

,Reporting Economy,Product/Sector,Partner Economy,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,World,SI3_AGG - TO - Total merchandise,World,15438092.0,18438364.0,18657296.0,18966119.0,19060809.0,16733507.0,16211194.0,17985896.0,19836342.0,19284167.0,17812107.0
1,World,SI3_AGG - AG - Agricultural products,World,1391529.0,1701893.0,1681611.0,1756648.0,1798003.0,1594510.0,1599989.0,1759978.0,1851346.0,1823105.0,NaN
2,World,SI3_AGG - AGFO - Food,World,1144082.0,1387912.0,1391766.0,1465158.0,1510854.0,1340065.0,1354624.0,1482990.0,1551785.0,1547817.0,NaN
3,World,SI3_AGG - MI - Fuels and mining products,World,3183852.0,4278365.0,4227403.0,4208130.0,3874299.0,2489603.0,2124334.0,2755203.0,3408176.0,3168025.0,NaN
4,World,SI3_AGG - MIFU - Fuels,World,2462969.0,3361172.0,3401742.0,3403308.0,3091267.0,1859835.0,1528358.0,2006039.0,2570707.0,2350244.0,NaN


#### Treatment of Missing Values

In [41]:
# Checking for missing values
dropped_rows_df.isnull().sum()

Reporting Economy       0
Product/Sector          0
Partner Economy         0
2010                  147
2011                  111
2012                  157
2013                  177
2014                  188
2015                  172
2016                  186
2017                  153
2018                  152
2019                  152
2020                 3199
dtype: int64

In [42]:
# Dealing with missing values with the fillna function
dropped_rows_df = dropped_rows_df.fillna(0)

In [45]:
display(dropped_rows_df.isna().sum())
print('----------')
display('Now all of our empty values have been successfully filled with 0')

Reporting Economy    0
Product/Sector       0
Partner Economy      0
2010                 0
2011                 0
2012                 0
2013                 0
2014                 0
2015                 0
2016                 0
2017                 0
2018                 0
2019                 0
2020                 0
dtype: int64

----------


'Now all of our empty values have been successfully filled with 0'

In [ ]:
# If there are any missing values, how do you plan to treat those data columns?

In [ ]:
# Perform any transformation on the columns in the dataset to enable further analysis.

#### Final Dataframe

In [ ]:
# Change data type to int

In [ ]:
# Change date type
# display(dropped_rows_df.dtypes)

# Change Float to DateTime(Year)
# to_dt_y = dropped_rows_df.iloc[:,3:]

<br><br>

<h1 align="center"> Analytical Transformations </h1>

<h1 align="center"> Data Analysis </h1>

#### Descriptive Statistical Analysis

In [ ]:
# Using basic statistical measures such as measurements of central tendancy such as mean, median and mode.

#### Distribution of Variables

In [ ]:
# Identify the distribution of the data to understand the range of values and how the data is structured.

#### Outliers in the dataset

In [ ]:
# Identify if there are any outliers in the dataset based on statistical measures.

<br><br>

<h1 align="center"> Reflections </h1>

#### Summary of Data Analysis

In [ ]:
# What insights should the user takeaway from EDA.

#### Questions unanswered

In [ ]:
# What aspects of the research question were we unable to answer and why?

#### Recommendations

In [ ]:
# Identify if there are any outliers in the dataset based on statistical measures.

#### Next Steps

In [ ]:
# What will the analyst do next based on the analysis?